Import the necessary python packages and libraries in the following cell.

In [ ]:
## Import the libraries here in this cell

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim



In [ ]:
# Your code here
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)



Torch version: 2.1.0+cu121
Torchvision version: 0.16.0+cu121


In [ ]:
# device = // Your code here

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

*   Normalize data
*   Resize each image to 224 x 224
*   Add Color Jitter with hue and saturation as 0.5
*   Introduce Random horizontal flips
*   Rotate images about 20 degrees


In [ ]:
# transform = // Your code here

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(hue=0.5,saturation=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

Train and test Resnet18 on Even numbered Classes in CIFAR10 Dataset.

In [ ]:
# Filter only even-numbered classes

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Your code here
#HINT: Use torch utils Subset class to create a subset of the dataset

even_classes = [i for i in range(0,10,2)]

trainset_even = Subset(trainset,[i for i in range(len(trainset)) if trainset.targets[i] in even_classes])
testset_even = Subset(testset, [i for i in range(len(testset)) if testset.targets[i] in even_classes])

100%|██████████| 170498071/170498071 [00:06<00:00, 28342736.43it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
model = models.resnet18(weights='DEFAULT').to(device)
model.fc = nn.Linear(model.fc.in_features, len(even_classes)).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 54.2MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Complete the code for backpropagation inside the train function.


In [ ]:
def train(model, trainloader, criterion, optimizer, device):

    model.train()

    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in trainloader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Complete the code
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

        _, pred = outputs.max(1)
        total_train += labels.size(0)
        correct_train += pred.eq(labels).sum().item()

    train_acc = 100 * correct_train / total_train

    return train_loss/len(trainloader), train_acc

Complete the test function to test out the model.

In [ ]:
def test(model, testloader, criterion, device):
    model.eval()
    correct_test = 0
    total_test = 0

    # Your code here
    with torch.no_grad():
      for inputs,labels in testloader:
        inputs,labels = inputs.to(device),labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs,labels)

        _,pred = outputs.max(1)
        total_test +=labels.size(0)
        correct_test+=pred.eq(labels).sum().item()
    test_accuracy = 100 * correct_test / total_test

    return test_accuracy

 Complete the code to plot the train and test accuracies

In [ ]:
def plot_accuracies(train_accuracies, test_accuracies, epochs):

    # Your code here
    plt.plot(range(1,epochs+1),train_accuracies,label='Train Accuracy')
    plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy')

    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracies vs Epochs')
    plt.legend()
    plt.show()


Complete the code to plot a Confusion Matrix (without sklearn)

In [ ]:
def plot_confusion_matrix(model, dataloader, class_names, device):
    model.eval()
    all_labels = []
    all_predictions = []

    num_classes = len(class_names)
    confusion_matrix = torch.zeros(num_classes, num_classes)

    # Your code here
    with torch.no_grad():
      for inputs,labels in dataloader:
        inputs,labels = inputs.to(device),labels.to(device)
        outputs = model(inputs)
        _,predictions = outputs.max(1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

    confusion_matrix = confusion_matrix(all_labels,all_predictions)

    plt.figure(figsize=(num_classes, num_classes))
    sns.heatmap(confusion_matrix, annot=True, fmt=".0f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


Complete the following code block to train and test the model.

In [ ]:
epochs = 10
train_accuracies = []
test_accuracies = []

for epoch in range(epochs):

        # Complete the loop
        train_loss ,train_acc = train(model,trainloader,criterion,optimizer,device)
        train_accuracies.append(train_acc)

        test_acc  = test(model, testloader, criterion, device)
        test_accuracies.append(test_acc)

        print(f'Epoch {epoch + 1} | Train Loss: {train_loss:.3f} | Train Accuracy: {train_acc:.3f}% | Test Accuracy: {test_acc:.3f}%')


plot_accuracies(train_accuracies, test_accuracies, epochs)
class_names = ['0', '2', '4', '6', '8']
plot_confusion_matrix(model, testloader, class_names, device)